In [0]:
# Create UC volumes for raw landing and checkpoints
spark.sql("CREATE VOLUME IF NOT EXISTS workspace.damg7370.mm_raw COMMENT 'Raw landing for material master'")
spark.sql("CREATE VOLUME IF NOT EXISTS workspace.damg7370.mm_ckpt COMMENT 'Checkpoints for streams'")

# Show them
spark.sql("SHOW VOLUMES IN workspace.damg7370").show(truncate=False)


+--------+-----------+
|database|volume_name|
+--------+-----------+
|damg7370|mm_ckpt    |
|damg7370|mm_raw     |
+--------+-----------+



In [0]:
bronze_table_path = "/Volumes/workspace/damg7370/mm_raw/bronze/material_master"
bronze_checkpoint = "/Volumes/workspace/damg7370/mm_ckpt/bronze/material_master"

dbutils.fs.rm(bronze_table_path, recurse=True)
dbutils.fs.rm(bronze_checkpoint, recurse=True)

print("Old Bronze data and checkpoint deleted.")


Old Bronze data and checkpoint deleted.


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import current_timestamp, col

raw_path = "/Volumes/workspace/damg7370/mm_raw/material_master/from_table"
bronze_table_path = "/Volumes/workspace/damg7370/mm_raw/bronze/material_master"
bronze_checkpoint = "/Volumes/workspace/damg7370/mm_ckpt/bronze/material_master"

schema = StructType([
    StructField("material_id", StringType(), True),
    StructField("material_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("sub_category", StringType(), True),
    StructField("uom", StringType(), True),
    StructField("unit_cost", DoubleType(), True),
    StructField("supplier_name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("plant", StringType(), True),
    StructField("status", StringType(), True),
    StructField("last_updated", StringType(), True),
    StructField("lead_time_days", IntegerType(), True),
    StructField("safety_stock", IntegerType(), True),
    StructField("reorder_level", IntegerType(), True),
    StructField("remarks", StringType(), True),
])

raw_stream = (
    spark.readStream
        .format("csv")
        .option("header", True)
        .option("delimiter", "|")
        .option("sep", "|")
        .option("quote", "\u0000")
        .schema(schema)
        .load(raw_path)
        .withColumn("bronze_ingest_ts", current_timestamp())
        .withColumn("source_file", col("_metadata.file_path"))
)

q = (
    raw_stream.writeStream
        .format("delta")
        .option("checkpointLocation", bronze_checkpoint)
        .outputMode("append")
        .trigger(once=True)
        .start(bronze_table_path)
)
q.awaitTermination()

display(spark.read.format("delta").load(bronze_table_path).limit(20))


material_id,material_name,category,sub_category,uom,unit_cost,supplier_name,country,plant,status,last_updated,lead_time_days,safety_stock,reorder_level,remarks,bronze_ingest_ts,source_file
"""M00001",Metal Chair,Raw Material,Metal,L,70.63,"Rodriguez""",null,null,null,null,null,null,null,null,2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00002",Plastic All,Raw Material,Plastic,NOS,14.24,Garza Inc,Germany,PlantD,Active,2025-08-21,16,653,167,"Discussion own night.""",2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00003",Plastic Perform,Raw Material,Plastic,MTR,78.58,"Santos""",null,null,null,null,null,null,null,null,2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00004",Paint Source,Consumable,Paint,KG,268.58,Peterson-Moore,Japan,PlantA,Obsolete,2025-09-13,11,899,346,"Behavior benefit.""",2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00005",Coolant Show,Consumable,Coolant,KG,114.72,Munoz-Roman,India,PlantA,Active,2025-09-25,5,439,167,"By two bad fall pick.""",2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00006",Tape Answer,Packaging,Tape,NOS,105.54,Ray-Bush,India,PlantA,Obsolete,2025-10-15,7,596,398,"Significant world talk.""",2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00007",Brake Prove,Component,Brake,MTR,494.77,James Group,Brazil,PlantB,Obsolete,2025-10-31,12,913,418,"Class great prove reduce.""",2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00008",Plastic Shoulder,Raw Material,Plastic,KG,201.18,"Wong""",null,null,null,null,null,null,null,null,2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00009",Wrap Stand,Packaging,Wrap,MTR,70.68,"Hawkins""",null,null,null,null,null,null,null,null,2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv
"""M00010",Tape Under,Packaging,Tape,NOS,70.03,Clark PLC,Japan,PlantA,Active,2025-10-20,5,206,346,"Defense material those poor.""",2025-11-10T23:45:26.043Z,dbfs:/Volumes/workspace/damg7370/mm_raw/material_master/from_table/part-00000-tid-2760754592554938358-fa59a57a-ab9a-4edd-8b1d-17914a4c5037-127-1-c000.csv


In [0]:
%sql
SELECT material_id, material_name, category, unit_cost
FROM delta.`/Volumes/workspace/damg7370/mm_raw/bronze/material_master`
LIMIT 10;


material_id,material_name,category,unit_cost
"""M00001",Metal Chair,Raw Material,70.63
"""M00002",Plastic All,Raw Material,14.24
"""M00003",Plastic Perform,Raw Material,78.58
"""M00004",Paint Source,Consumable,268.58
"""M00005",Coolant Show,Consumable,114.72
"""M00006",Tape Answer,Packaging,105.54
"""M00007",Brake Prove,Component,494.77
"""M00008",Plastic Shoulder,Raw Material,201.18
"""M00009",Wrap Stand,Packaging,70.68
"""M00010",Tape Under,Packaging,70.03
